# The following function, calculate the Score_Ratio.

*   we have Score_Ratio=Score_mean/Score_sd
*   The higher  Score_Ratio shows that it has the highest mean with the lowest standard deviation
*   It add one column name Score_ratio per score to the table



In [ ]:
import pandas as pd

def calculate_ratio_metrics(df):
    # Define the list of metrics for which you want to calculate ratios
    metrics = ['cos', 'jaccard', 'precision', 'lev', 'bleu', 'rouge',
               'recall', 'f1', 'sorensen', 'exact_match', 'iou']

    # Calculate ratio of mean to sd for each metric and add as new columns
    for metric in metrics:
        mean_col = f"{metric}_mean"
        sd_col = f"{metric}_sd"
        ratio_col = f"{metric}_ratio"

        # Calculate ratio (handle division by zero)
        df[ratio_col] = df[mean_col] / df[sd_col].replace(0, 1e-9)  # Replace 0 sd with a very small number

    # Return the updated DataFrame with ratios
    return df


# **The following code ask you the "entity name" and "score name" and give you the best LLM based on the score_ratio for that specific entity.**

In [ ]:


# Step 1: Ask for file path
file_path = input("Enter the file path of the CSV file: ")

# Step 2: Read CSV into a DataFrame
try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    print("File not found. Please check the file path and try again.")
    exit()

# Step 3: Perform ratio metrics calculation if needed
df = calculate_ratio_metrics(df)

# Step 4: Ask for score name
score_name = input("Enter the name of the score_ratio column: ")

# Step 5: Ask for entity name
entity = input("Enter the entity name: ")

# List of LLM names to iterate over
llm_names = ['aya', 'mistral', 'llama3', 'phi3', 'qwen2', 'gemma']  # Add more LLM names as needed

# Initialize variables to track highest values and corresponding LLM
max_values = {}
for llm_name in llm_names:
    max_values[llm_name] = {'value': -float('inf'), 'row': None}

# Iterate over each LLM name
for llm_name in llm_names:
    # Filter rows based on LLM name and entity
    filtered_df = df[(df['LLM'] == llm_name) & (df['entity'] == entity)]

    # Check if filtered_df is empty
    if filtered_df.empty:
        print(f"No data found for LLM '{llm_name}' and entity '{entity}'.")
        continue

    # Check for highest value in score column within the filtered rows
    max_value = filtered_df[score_name].max()
    max_row = filtered_df[filtered_df[score_name] == max_value]

    # Update max_values dictionary if current max_value is higher
    if max_value > max_values[llm_name]['value']:
        max_values[llm_name]['value'] = max_value
        max_values[llm_name]['row'] = max_row

# Print results for each LLM
for llm_name in llm_names:
    if max_values[llm_name]['value'] == 0:
        print("")
        print("*************************************************************************************")
        print("")
        # Print only one row if max_value is zero
        if max_values[llm_name]['row'] is not None:
            columns_to_print = df.columns[:4].tolist() + [score_name]
            print(f"Highest {score_name} score found for LLM '{llm_name}' and entity '{entity}': {max_values[llm_name]['value']}")
            print(max_values[llm_name]['row'][columns_to_print].head(1).to_string(index=False))  # Print only one row
    else:
        try:
            print("")
            print("*************************************************************************************")
            print("")
            # Print formatted output for non-zero max_value
            columns_to_print = df.columns[:4].tolist() + [score_name]
            print(f"Highest {score_name} score found for LLM '{llm_name}' and entity '{entity}': {max_values[llm_name]['value']}")
            print(max_values[llm_name]['row'][columns_to_print].to_string(index=False))
        except KeyError:
            print(f"Column '{score_name}' not found. Please check the column name and try again.")

# Compare highest values of different LLMs and print the comparison
highest_llm = None
highest_value = -float('inf')
print("*************************************************************************************")
print("")
print(f"Which LLM is the best for '{entity}' based on {score_name}?")
print("")
for llm_name in llm_names:
    if max_values[llm_name]['value'] > highest_value:
        highest_value = max_values[llm_name]['value']
        highest_llm = llm_name

if highest_llm:
    print(f"The highest {score_name}  for entity '{entity}' is  for LLM '{highest_llm}' with a value of {highest_value}.")
    print("")
else:
    print(f"No valid data found for entity '{entity}' among the specified LLMs.")

#EXAMPLE:
# Enter the file path of the CSV file: /content/scores_mean_results (1).csv
# Enter the name of the score_ratio column: iou_ratio
# Enter the entity name: Crew Information and People on Board

Enter the file path of the CSV file: /content/scores_mean_results (1).csv
Enter the name of the score_ratio column: iou_ratio
Enter the entity name: Crew Information and People on Board

*************************************************************************************

Highest iou_ratio score found for LLM 'aya' and entity 'Crew Information and People on Board': 0.6945568211602761
 Unnamed: 0 LLM                               entity                                                                                                                                   setting  iou_ratio
        443 aya Crew Information and People on Board settings:chunk_size:1000,chunk_overlap:250,embed_model:nomic-embed-text,llm:aya,temperature:Unknown,top_k:Unknown,top_p:0.5,verbose:False   0.694557

*************************************************************************************

Highest iou_ratio score found for LLM 'mistral' and entity 'Crew Information and People on Board': 0.7138127735504266
 U